<a href="https://colab.research.google.com/github/voodoohop/pollinations/blob/master/colabs/pollinator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Code (Frontend and IPFS Connection)

In [118]:
#@title Install IPFS

!pip install coolname gdown papermill
#!pip install asyncio nest-asyncio coolname aioipfs
#!pip install ipfshttpclient
#!npm install -g nodemon
!npm install -g voodoohop/pollinations/app


%cd /content
!git clone https://github.com/voodoohop/pollinations.git
%cd /content/pollinations/app
!git pull
!git checkout dev
!git pull
!npm install
!npm install -g
%cd -
#!npm install forever -g

npm ERR! code ENOLOCAL
npm ERR! Could not install from "voodoohop/pollinations/app" as it does not contain a package.json file.

npm ERR! A complete log of this run can be found in:
npm ERR!     /root/.npm/_logs/2021-05-17T13_53_41_794Z-debug.log
/content
Cloning into 'pollinations'...
remote: Enumerating objects: 1741, done.
remote: Counting objects: 100% (133/133), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 1741 (delta 77), reused 64 (delta 35), pack-reused 1608
Receiving objects: 100% (1741/1741), 15.65 MiB | 21.43 MiB/s, done.
Resolving deltas: 100% (1120/1120), done.
/content/pollinations/app
Already up to date.
Branch 'dev' set up to track remote branch 'dev' from 'origin'.
Switched to a new branch 'dev'
Already up to date.
npm WARN deprecated core-js@2.6.12: core-js@<3.3 is no longer maintained and not recommended for usage due to the number of issues. Because of the V8 engine whims, feature detection in old core-js versions could cause a slowdown up to

In [82]:
#@title Config
from coolname import generate_slug
ipfs_nodeid = generate_slug(2)#asyncio.run(loop.create_task(get_node_id()))
IPFS_PEER = "ipfs.pollinations.ai"
ipfs_root = "/content/ipfs"

print(f"---Settings---\nNODE_ID: {ipfs_nodeid}\nIPFS_ROOT: {ipfs_root}\nIPFS_PEER: {IPFS_PEER}")

---Settings---
NODE_ID: neon-beluga
IPFS_ROOT: /content/ipfs
IPFS_PEER: ipfs.pollinations.ai


In [83]:
#@title Empty and create ipfs root: */content/ipfs*



!rm -r $ipfs_root
!mkdir -p $ipfs_root/input
!mkdir -p $ipfs_root/output
!echo "Created IPFS resource" > $ipfs_root/output/log

parameters={}
for key, default in parameters.items():
    print(default, file=open(f"{ipfs_root}/input/{key}", 'a'))



In [100]:
!wget https://dist.ipfs.io/go-ipfs/v0.8.0/go-ipfs_v0.8.0_linux-amd64.tar.gz
!tar -xvzf go-ipfs_v0.8.0_linux-amd64.tar.gz
%cd go-ipfs
!sudo bash install.sh
%cd -
!ipfs init
!
!nohup ipfs daemon --enable-namesys-pubsub --enable-pubsub-experiment &
!sleep 15
!ipfs swarm connect /ip4/18.157.205.205/tcp/4001/p2p/12D3KooW9rNKBCYKi585Dcj2wfLpETTkQSjrtRHHK8XfeJLvr5Ka



--2021-05-17 12:42:44--  https://dist.ipfs.io/go-ipfs/v0.8.0/go-ipfs_v0.8.0_linux-amd64.tar.gz
Resolving dist.ipfs.io (dist.ipfs.io)... 209.94.78.1, 2602:fea2:3::1
Connecting to dist.ipfs.io (dist.ipfs.io)|209.94.78.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25904633 (25M) [application/gzip]
Saving to: ‘go-ipfs_v0.8.0_linux-amd64.tar.gz.3’

go-ipfs_v0.8.0_linu 100%[===================>]  24.70M  10.7MB/s    in 2.3s    

2021-05-17 12:42:47 (10.7 MB/s) - ‘go-ipfs_v0.8.0_linux-amd64.tar.gz.3’ saved [25904633/25904633]

go-ipfs/install.sh
go-ipfs/ipfs
go-ipfs/LICENSE
go-ipfs/LICENSE-APACHE
go-ipfs/LICENSE-MIT
go-ipfs/README.md
/content/go-ipfs
Moved ./ipfs to /usr/local/bin
/content
Error: ipfs daemon is running. please stop it to run this command
Use 'ipfs init --help' for information about this command
nohup: appending output to 'nohup.out'
connect 12D3KooW9rNKBCYKi585Dcj2wfLpETTkQSjrtRHHK8XfeJLvr5Ka success


In [93]:
from os.path import join


notebook_path = join(ipfs_root,"latentvisions.ipynb")
notebook_params_path = join(ipfs_root,"parameters.json")

!gdown --id 1zOA0bUJ68zuZXcbCoN2LPNjnkvxzFpv_ -O $notebook_path

!python get_params.py $notebook_path > $notebook_params_path

Downloading...
From: https://drive.google.com/uc?id=1zOA0bUJ68zuZXcbCoN2LPNjnkvxzFpv_
To: /content/ipfs/latentvisions.ipynb
100% 20.2k/20.2k [00:00<00:00, 34.7MB/s]


In [142]:

content_id=!ipfs add -r -Q $ipfs_root
content_id=content_id[0]
node_id=!ipfs name publish -Q --offline /ipfs/$content_id
node_id=node_id[0]

#!ipfs pubsub pub $node_id $content_id
print(content_id,node_id)

QmQZNKrQK9x1L6sXCmXuLKfztAPjQqHdPsRMo1BgN8cjdx k51qzi5uqu5dh0esiimwcc251a5oq9ivhjppf1zqip1cpdjcvtwj1olbdmqk9k


In [148]:
!./ipfs_sub_one.sh $node_id 

7941 7943
IPFS_PID 7942
./ipfs_sub_one.sh: line 19: wait: pid 7941 is not a child of this shell
DONE


In [102]:
#@title Create connection to frontend UI

import IPython
from google.colab import output

display(IPython.display.Javascript(f'''
  const iframe = document.createElement("iframe")
  iframe.src = `http://localhost:3000/incolab.html?node={node_id},{content_id}`

  document.querySelector("#output-area").appendChild(iframe)
'''))



<IPython.core.display.Javascript object>

In [63]:
async def update_ipfs_content(contentid):
  global ipfs_contentid
  contentid = contentid.decode("utf-8")
  
  await client.get(contentid)
  !cp -rv $contentid/* $ipfs_root
  #!rm -r $contentid
  ipfs_contentid = contentid

content_id=!ipfs name resolve k51qzi5uqu5dh0esiimwcc251a5oq9ivhjppf1zqip1cpdjcvtwj1olbdmqk9k
#on_frontend_data(update_ipfs_content)
print(content_id)

['/ipfs/QmQ26epDtE4Bi1TJdUyeEy7F4tA6JGGM7bWycbMq1sp6QP']


In [ ]:
from glob import glob
from os.path import basename
for path in glob(f"{ipfs_root}/input/*"):
  key = basename(path)
  with open(path,'r') as f:
    value = f.read()
    parameters[key] = value

print("New parameters", parameters)

## DISABLED! CODE (DEEP-DAZE)


DISABLED

```
from subprocess import Popen, PIPE, STDOUT
from sys import stdout
from glob import glob
import os
import json
import itertools
from base64 import b64encode
from time import sleep
import requests
import os
from mimetypes import guess_type 
import re
flatten = itertools.chain.from_iterable

#parameters["text"] = "A black and white photo of Saeko with her new Elektron FM synthesizer"
def run(text, args, watch_media_paths=["./*.jpg","./*.mp4"]):
  print("Running",text)
  for to_remove in watch_media_paths:
    for filepath in glob(to_remove):
      print("Deleting",filepath,"before starting.")
      os.remove(filepath)

  arguments = args.split(" ")
  cmd = ['imagine'] + arguments +[text]
  
  p = Popen(cmd, stdout = PIPE, 
          stderr = STDOUT, shell = True)

  found_media_files=[]
  
  while True:
    line = p.stdout.readline()#
    if not line: break

    stdout.write(line)
    with open(f"{ipfs_root}/log","w") as log_out:
      log_out.write(line.decode("utf-8"))
      log_out.flush()
    #headers = { 
    #    "params": params,
    #    "text": text
    #}
    #conn.send(body=line.decode("utf-8"), headers=headers, destination='/topic/colabOut')
    #print("new contentid", client.add(ipfs_root, recursive=True))
    
    #continue
    new_found_media_files = get_matching_files(watch_media_paths)
    new_found_media_files.sort(key=os.path.getmtime)
    added_media_files = list(set(new_found_media_files)-set(found_media_files))
    found_media_files = new_found_media_files
    pin=False
    if len(added_media_files) > 0:
      for added in added_media_files:
        sleep(0.1)
        if len(added) > 0:
          !cp $added $ipfs_root/output
          print("Adding:", added,"to ipfs root")
          pin=True
    add_and_publish()    
       

def get_matching_files(search_paths):
  return list(flatten([glob(search_path) for search_path in search_paths]))

run(**parameters)

sleep(1000000)
```



# New section

In [ ]:
#!pip install --upgrade pip

#!pip install --upgrade ipython
#import papermill as pm
asda
from subprocess import Popen, PIPE, STDOUT
from sys import stdout
import os

#parameters["text"] = "A black and white photo of Saeko with her new Elektron FM synthesizer"
def run(parameters, watch_media_paths="./*.jpg"):
  print("Running",parameters)
 
  for filepath in glob(watch_media_paths):
    print("Deleting",filepath,"before starting.")
    os.remove(filepath)

  _params = [["-p", key, value] for key, value in parameters.items()]
  params = [str(val) for sublist in _params for val in sublist]
  #%run latentvisions.ipynb
  #!papermill "./latentvisions.ipynb" ./output 
  cmd = ["papermill", "/content/latentvisions.ipynb","/content/latentvisions_output.ipynb"] + params
  print(" ".join(cmd))
  p = Popen(cmd, stdout = PIPE, 
          stderr = STDOUT, shell = False)

  found_media_files=[]

  while True:
    line = p.stdout.readline()#
    if not line: break

    stdout.write(line)
    with open(f"{ipfs_root}/log","w") as log_out:
      log_out.write(line.decode("utf-8"))
      log_out.flush()
      
    #headers = { 
    #    "params": params,
    #    "text": text
    #}
    #conn.send(body=line.decode("utf-8"), headers=headers, destination='/topic/colabOut')
    #print("new contentid", client.add(ipfs_root, recursive=True))
    
    #continue
    new_found_media_files = glob(watch_media_paths)
    new_found_media_files.sort(key=os.path.getmtime)
    added_media_files = list(set(new_found_media_files)-set(found_media_files))
    found_media_files = new_found_media_files
    pin=False
    if len(added_media_files) > 0:
      for added in added_media_files:
        sleep(0.1)
        if len(added) > 0:
          !cp $added $ipfs_root/output
          print("Adding:", added,"to ipfs root")
          pin=True
    add_and_publish()    
        
while True:
  run(parameters,"./taming_transformers/*.jpg")

     |████████████████████████████████| 143kB 7.8MB/s 
     |████████████████████████████████| 747kB 38.4MB/s 
     |████████████████████████████████| 727kB 22.6MB/s 
     |████████████████████████████████| 122kB 44.1MB/s 
  Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Found existing installation: jupyter-client 5.3.5
    Uninstalling jupyter-client-5.3.5:
      Successfully uninstalled jupyter-client-5.3.5


Usage: papermill [OPTIONS] NOTEBOOK_PATH
                 [OUTPUT_PATH]

Parameters inferred for notebook './latentvisions.ipynb':
  text_input: Unknown type (default "A digital render in dark blue of a machine-based software intelligence capable of evolving autonomously as more data is added into its algorithmic structure.")
                                  @param {type: "string"}
  text_not: Unknown type (default 'disconnected, confusing, incoherent, text, letters')
                                  @param {type: "string"}
  num_iterations: Unknown type (default 1000)
                                  @param {type: "number"}


NameError: ignored